In [568]:
# Imports
import numpy as np
import pandas as pd
import datetime

In [569]:
#data = pd.concat([pd.read_csv('Data/2009.csv'), pd.read_csv('Data/2010.csv'), 
 #                 pd.read_csv('Data/2011.csv'), pd.read_csv('Data/2012.csv'), 
  #                pd.read_csv('Data/2013.csv'), pd.read_csv('Data/2014.csv'), 
   #               pd.read_csv('Data/2015.csv'), pd.read_csv('Data/2016.csv'), 
    #              pd.read_csv('Data/2017.csv'), pd.read_csv('Data/2018.csv')])

# Read 2018 data only
data = pd.read_csv('Data/2010.csv')

In [570]:
# Convert string to DateTime and set as index
data.FL_DATE = pd.to_datetime(data.FL_DATE, infer_datetime_format=True)
data.set_index('FL_DATE', inplace=True)
data.drop(columns='Unnamed: 27', axis=1, inplace=True) # Drop last weird column


,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,...,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
FL_DATE,,,,,,,,,,,,,,,,,,,,,
2010-01-01,MQ,4438,LGA,RDU,1940,2101.0,81.0,14.0,2115.0,2225.0,...,0.0,110.0,106.0,70.0,431.0,0.0,0.0,0.0,0.0,77.0
2010-01-01,MQ,4440,DCA,JFK,950,949.0,-1.0,13.0,1002.0,1040.0,...,0.0,70.0,60.0,38.0,213.0,NaN,NaN,NaN,NaN,NaN
2010-01-01,MQ,4443,LGA,RDU,900,937.0,37.0,11.0,948.0,1105.0,...,0.0,120.0,91.0,77.0,431.0,NaN,NaN,NaN,NaN,NaN
2010-01-01,MQ,4445,RDU,LGA,1940,1935.0,-5.0,12.0,1947.0,2051.0,...,0.0,100.0,80.0,64.0,431.0,NaN,NaN,NaN,NaN,NaN
2010-01-01,MQ,4447,JFK,DCA,2035,2236.0,121.0,18.0,2254.0,2341.0,...,0.0,85.0,66.0,47.0,213.0,0.0,0.0,0.0,0.0,102.0


In [571]:
# Drop NaN values in the variables we will use in our baseline model
data = data[data['TAXI_OUT'].notna()]
data = data[data['ORIGIN'].notna()]
data = data[data['DEST'].notna()]


In [580]:
# Take first 6 months
df = data.loc['2010-01-01':'2010-07-01']


In [581]:
# Renaming airline codes to company names
# Source: https://en.wikipedia.org/wiki/List_of_airlines_of_the_United_States

df['OP_CARRIER'].replace({
    'UA':'United Airlines',
    'AS':'Alaska Airlines',
    '9E':'Endeavor Air',
    'B6':'JetBlue Airways',
    'EV':'ExpressJet',
    'F9':'Frontier Airlines',
    'G4':'Allegiant Air',
    'HA':'Hawaiian Airlines',
    'MQ':'Envoy Air',
    'NK':'Spirit Airlines',
    'OH':'PSA Airlines',
    'OO':'SkyWest Airlines',
    'VX':'Virgin America',
    'WN':'Southwest Airlines',
    'YV':'Mesa Airline',
    'YX':'Republic Airways',
    'AA':'American Airlines',
    'DL':'Delta Airlines'
},inplace=True)

In [582]:
df = df[df.ORIGIN == 'JFK']

In [583]:
# Take only the delayed flights
df = df[df['CARRIER_DELAY' ] >= 0]

In [584]:
delays = df[['CARRIER_DELAY', 'DEP_TIME']]
delays['DEP_TIME'] = delays['DEP_TIME'].astype(int)

preps = []
for i in range(len(delays['DEP_TIME'])):
    # Zero fill values
    dep_time_val = str(delays['DEP_TIME'].iloc[i]).zfill(4)
    # If flight at 24:00, set that as 00:00
    if dep_time_val == str(2400):
        dep_time_act = datetime.datetime.strptime('0000','%H%M').strftime('%H:%M')
    else:
        dep_time_act = datetime.datetime.strptime(dep_time_val,'%H%M').strftime('%H:%M')
    

    # append
    preps.append(dep_time_act)

# Drop and add corrected times
df.drop(columns=['DEP_TIME'])
df['DEP_TIME'] = preps
df['DEP_TIME'] = pd.to_datetime(df['DEP_TIME'], format='%H:%M') # Convert to datetime

# Convert FL time to column
df = df.reset_index(level=0)

/var/folders/fs/10_w6dn131d7nmw8myr18l6r0000gn/T/ipykernel_37927/2885194547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  delays['DEP_TIME'] = delays['DEP_TIME'].astype(int)


In [585]:
comb_date = []
# Loop through all to combine dates and time
for i in range(len(df['DEP_TIME'])):

    # Get date and time
    date = datetime.datetime.date(df['FL_DATE'].iloc[i])
    time = datetime.datetime.time(df['DEP_TIME'].iloc[i])

    # Get combined as a string
    comb = datetime.datetime.combine(date, time).strftime('%Y-%m-%d %H:%M:%S')

    #append
    comb_date.append(comb)


In [586]:
# Add column, and remove DEP_TIME and FL_DATE
df.drop(columns=['FL_DATE', 'DEP_TIME'], inplace=True)
df['DATE_TIME'] = pd.to_datetime(comb_date)

In [587]:
# Move it to the front of the data frame
dates = df.pop('DATE_TIME')
df.insert(0, 'DATE_TIME', dates)

### WEATHER DATA FOR JFK

In [589]:
# Load weather data
df_weather_all = pd.read_csv('Data/Weather.csv')
# Select only relevant features
df_weather = df_weather_all[["NAME","DATE","HLY-TEMP-NORMAL","HLY-WIND-AVGSPD"]]
# Select only JFK airport and from January 1st to July 1st
df_weather= df_weather[(df_weather["NAME"] == "JFK INTERNATIONAL AIRPORT, NY US") & (df_weather["DATE"] < '07-01T00:00:00')]

In [590]:
# Compare shape to see effect of preprocessing
print('Shape before:',data.shape,'\n Shape after:',df.shape)

Shape before: (6338168, 26) 
 Shape after: (10900, 26)


In [361]:
# Save as new dataframe
df.to_csv('Data/data_pre.csv')
df_weather.to_csv('Data/weather_pre.csv')